<a href="https://colab.research.google.com/github/talal777-ye/for-Collage/blob/main/IoT_Security_%3CTalalYeslamBadubbah%3E_%3C22202041052%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import random
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend

# --- Step 1: Configuration & Setup ---
# IoT devices often use a pre-shared key (PSK)
# We generate a 256-bit key for AES-256
SECRET_KEY = os.urandom(32)
# Initialization Vector (IV) ensures unique encryption for the same data
IV = os.urandom(16)

def generate_sensor_data():
    """Step 1: Generate random temperature and humidity readings."""
    data = {
        "sensor_id": "IOT_SENSOR_01",
        "temperature": round(random.uniform(20.0, 35.0), 2),  # Random temp 20-35 C
        "humidity": round(random.uniform(30.0, 80.0), 2)      # Random humidity 30-80%
    }
    return data

def encrypt_data(key, iv, plaintext_data):
    """Step 2: Encrypt data using AES (CBC Mode)."""
    # 1. Convert JSON data to bytes
    json_bytes = json.dumps(plaintext_data).encode('utf-8')

    # 2. AES requires data to be a multiple of blocks (128-bit). We must pad the data.
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(json_bytes) + padder.finalize()

    # 3. Initialize Cipher
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # 4. Encrypt
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext

def decrypt_data(key, iv, ciphertext):
    """Step 3: Decrypt data on the receiver side."""
    # 1. Initialize Cipher
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # 2. Decrypt
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    # 3. Unpad the data to get original JSON bytes
    unpadder = padding.PKCS7(128).unpadder()
    plaintext_bytes = unpadder.update(padded_plaintext) + unpadder.finalize()

    # 4. Convert bytes back to JSON dictionary
    return json.loads(plaintext_bytes.decode('utf-8'))

# --- Main Simulation Execution ---

print("--- IoT Device Data Encryption Simulation ---\n")

# 1. Generate Data
sensor_reading = generate_sensor_data()
print(f"1. [Sensor Node] Generated Readings: {sensor_reading}")

# 2. Encrypt Data
encrypted_bytes = encrypt_data(SECRET_KEY, IV, sensor_reading)
# We encode in Base64 just to make it readable for printing
encrypted_b64 = base64.b64encode(encrypted_bytes).decode('utf-8')

print(f"2. [Sensor Node] Encrypting data using AES-256...")
print(f"   Key (Hidden): {base64.b64encode(SECRET_KEY).decode('utf-8')[0:10]}...")
print(f"   Ciphertext (Sent to Server): {encrypted_b64}")

print("\n--- Simulating Transmission ---\n")

# 3. Decrypt Data (Receiver Side)
try:
    decrypted_reading = decrypt_data(SECRET_KEY, IV, encrypted_bytes)
    print(f"3. [Base Station] Data Received & Decrypted Successfully.")
    print(f"4. [Base Station] Final Output: {decrypted_reading}")

    # Verification
    if sensor_reading == decrypted_reading:
        print("\n[SUCCESS] Integrity verified: Data matches original readings.")
    else:
        print("\n[ERROR] Integrity failed.")

except Exception as e:
    print(f"Decryption Failed: {e}")

--- IoT Device Data Encryption Simulation ---

1. [Sensor Node] Generated Readings: {'sensor_id': 'IOT_SENSOR_01', 'temperature': 26.86, 'humidity': 38.15}
2. [Sensor Node] Encrypting data using AES-256...
   Key (Hidden): z0egdxiy9t...
   Ciphertext (Sent to Server): IBbk5dyCozvAaEk6SlyHwYJmR8IkkdpmjaOYtRklrobghjs96LCO9QFx9siC8aYQi9nLmgO5sMqAuXq5YV/pAeB3mABZkN8YGT7MWXxL9tw=

--- Simulating Transmission ---

3. [Base Station] Data Received & Decrypted Successfully.
4. [Base Station] Final Output: {'sensor_id': 'IOT_SENSOR_01', 'temperature': 26.86, 'humidity': 38.15}

[SUCCESS] Integrity verified: Data matches original readings.


In [2]:
import time
import random
import hashlib
import os
from datetime import datetime

class IoTSecurityLifecycle:
    def __init__(self, device_id):
        self.device_id = device_id
        self.firmware_version = "1.0.0"
        self.device_keys = None # Storage for secure keys
        self.boot_signature = "valid_signature_hash_123" # Mock hardware root of trust

    def log(self, stage, message, status="INFO"):
        """Helper to print timestamped logs."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] [{stage.upper()}] {message} ... [{status}]")
        time.sleep(1) # Simulating processing time

    # --- Stage A: Threat Modeling ---
    def stage_threat_modeling(self):
        print(f"\n--- Stage A: Threat Modeling & Design ---")
        self.log("Threat Model", "Identifying potential attack vectors (Spoofing, Tampering)", "START")

        threats = ["DDoS Attack", "Man-in-the-Middle", "Physical Tampering", "Fake Firmware"]
        for threat in threats:
            print(f"    > Analysing mitigation for: {threat}")
            time.sleep(0.5)

        self.log("Threat Model", "Security architecture defined", "COMPLETED")

    # --- Stage B: Secure Key Injection ---
    def stage_key_injection(self):
        print(f"\n--- Stage B: Secure Key Injection (Manufacturing) ---")
        self.log("Key Injection", "Generating unique device identity keys", "PROCESSING")

        # Simulate generating a 256-bit unique private key
        mock_key = os.urandom(32).hex()
        self.device_keys = {
            "private_key": mock_key,
            "public_cert": f"cert_{self.device_id}_v1"
        }

        print(f"    > Injected Root Key: {mock_key[:10]}******")
        self.log("Key Injection", "Keys written to Write-Once-Memory (WOM)", "SECURED")

    # --- Stage C: Secure Boot Initialization ---
    def stage_secure_boot(self):
        print(f"\n--- Stage C: Secure Boot Initialization (Startup) ---")
        self.log("Secure Boot", "Verifying bootloader signature", "CHECKING")

        # Simulate verifying the digital signature of the firmware
        current_hash = "valid_signature_hash_123" # Mock hash of current code

        if current_hash == self.boot_signature:
            print("    > Signature Match: Hardware Root of Trust verified.")
            self.log("Secure Boot", "OS Kernel loading allowed", "SUCCESS")
        else:
            self.log("Secure Boot", "Signature Mismatch! Halting system.", "FAILED")
            exit()

    # --- Stage D: OTA Firmware Update Check ---
    def stage_ota_update(self):
        print(f"\n--- Stage D: OTA Firmware Update (Maintenance) ---")
        self.log("OTA Update", f"Current Version: {self.firmware_version}. Checking for updates", "CONNECTING")

        # Simulate finding a new version
        new_version = "1.2.0"
        print(f"    > New firmware found: v{new_version}")

        # Verify signature of the new firmware before installing (Critical security step)
        self.log("OTA Security", "Verifying digital signature of new firmware packet", "VERIFYING")
        time.sleep(1.5)

        self.firmware_version = new_version
        self.log("OTA Update", f"Update applied. New Version: {self.firmware_version}", "SUCCESS")

    # --- Stage E: Secure Decommissioning ---
    def stage_decommissioning(self):
        print(f"\n--- Stage E: Secure Decommissioning (End of Life) ---")
        self.log("Decommission", "Initiating factory reset and data wipe", "WARNING")

        if self.device_keys:
            print(f"    > Wiping cryptographic keys: {self.device_keys['private_key'][:10]}...")
            self.device_keys = None # Wipe data

        print("    > Clearing user data partition...")
        print("    > Revoking device certificate from cloud...")

        self.log("Decommission", "Device creates are wiped. Safe to discard.", "COMPLETED")

# --- Main Simulation Execution ---
if __name__ == "__main__":
    # Initialize the mock device
    iot_device = IoTSecurityLifecycle(device_id="SENSOR_XJ9_2025")

    # Run the lifecycle stages sequentially
    iot_device.stage_threat_modeling()
    iot_device.stage_key_injection()
    iot_device.stage_secure_boot()
    iot_device.stage_ota_update()

    # Simulate device reaching end of life after some time
    time.sleep(2)
    iot_device.stage_decommissioning()


--- Stage A: Threat Modeling & Design ---
[2025-12-10 18:51:42] [THREAT MODEL] Identifying potential attack vectors (Spoofing, Tampering) ... [START]
    > Analysing mitigation for: DDoS Attack
    > Analysing mitigation for: Man-in-the-Middle
    > Analysing mitigation for: Physical Tampering
    > Analysing mitigation for: Fake Firmware
[2025-12-10 18:51:45] [THREAT MODEL] Security architecture defined ... [COMPLETED]

--- Stage B: Secure Key Injection (Manufacturing) ---
[2025-12-10 18:51:46] [KEY INJECTION] Generating unique device identity keys ... [PROCESSING]
    > Injected Root Key: d28aedbf59******
[2025-12-10 18:51:47] [KEY INJECTION] Keys written to Write-Once-Memory (WOM) ... [SECURED]

--- Stage C: Secure Boot Initialization (Startup) ---
[2025-12-10 18:51:48] [SECURE BOOT] Verifying bootloader signature ... [CHECKING]
    > Signature Match: Hardware Root of Trust verified.
[2025-12-10 18:51:49] [SECURE BOOT] OS Kernel loading allowed ... [SUCCESS]

--- Stage D: OTA Firmw